In [1]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [2]:
# LOAD ALL IMAGES:
dt = 5
dict = {}
for img in glob.glob(f"DATA/ephemeris sat/inclination zero/{dt} step size/*"):
    txt = img             # stringa
    t = txt.split('_')[1] # numero
    dict[t] = txt

In [3]:
# IMG1:
idx = 0
img1=cv2.imread(dict[str(idx+1)])
craters_det1 = detect(img1)
# IMG2:
idx = 1
img2=cv2.imread(dict[str(idx+1)])
craters_det2 = detect(img2)

In [4]:
# Pandas DataFrame:
df_craters_det1 = sort_mat(craters_det1)
df_craters_det2 = sort_mat(craters_det2)

In [12]:
QUERY1 = find_query(df_craters_det1)
QUERY2 = find_query(df_craters_det2)

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 
 |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% 


In [13]:
QUERY1.sort_values(by='Angle1')

,Angle1,Angle2,Angle3,ctr1,ctr2,ctr3,x1,y1,r1,x2,y2,r2,x3,y3,r3
14,0.656463,173.314796,6.028741,14.0,1.0,7.0,735.5,704.5,125.00,794.5,761.5,26.50,137.5,248.5,13.50
10,2.094027,3.897703,174.008271,10.0,4.0,14.0,49.0,652.0,74.00,290.0,654.0,24.00,735.5,704.5,125.00
7,2.655981,1.699381,175.644639,7.0,3.0,1.0,137.5,248.5,13.50,547.5,549.5,15.50,794.5,761.5,26.50
4,3.621659,173.331435,3.046906,4.0,3.0,5.0,290.0,654.0,24.00,547.5,549.5,15.50,85.0,766.5,33.75
19,16.211894,29.291150,134.496956,19.0,3.0,7.0,834.0,503.0,11.00,547.5,549.5,15.50,137.5,248.5,13.50
12,17.731574,126.820471,35.447955,12.0,1.0,5.0,573.5,606.5,11.00,794.5,761.5,26.50,85.0,766.5,33.75
11,27.098843,21.962630,130.938527,11.0,2.0,5.0,83.0,164.0,12.50,220.0,500.5,16.25,85.0,766.5,33.75
3,28.157060,31.180740,120.662200,3.0,2.0,11.0,547.5,549.5,15.50,220.0,500.5,16.25,83.0,164.0,12.50
1,31.406280,25.954683,122.639037,1.0,4.0,7.0,794.5,761.5,26.50,290.0,654.0,24.00,137.5,248.5,13.50
2,35.588421,116.636838,27.774741,2.0,3.0,7.0,220.0,500.5,16.25,547.5,549.5,15.50,137.5,248.5,13.50


In [14]:
QUERY2.sort_values(by='Angle1')

,Angle1,Angle2,Angle3,ctr1,ctr2,ctr3,x1,y1,r1,x2,y2,r2,x3,y3,r3
1,0.000000,NaN,NaN,1.0,1.0,8.0,68.5,248.5,14.50,68.5,248.5,14.50,152.5,499.5,16.50
3,0.000000,NaN,NaN,3.0,3.0,2.0,197.5,565.5,13.00,197.5,565.5,13.00,726.5,763.5,26.50
7,0.000000,NaN,NaN,7.0,7.0,9.0,28.0,768.5,32.25,28.0,768.5,32.25,812.0,451.5,12.25
8,3.639974,164.216526,12.143500,8.0,3.0,1.0,152.5,499.5,16.50,197.5,565.5,13.00,68.5,248.5,14.50
20,7.051135,4.582145,168.366719,20.0,3.0,7.0,515.5,312.5,64.00,197.5,565.5,13.00,28.0,768.5,32.25
11,8.839947,115.672492,55.487561,11.0,3.0,1.0,140.5,553.0,11.25,197.5,565.5,13.00,68.5,248.5,14.50
4,16.229949,20.655552,143.114499,4.0,6.0,11.0,696.0,708.5,139.25,479.5,549.5,16.00,140.5,553.0,11.25
6,20.084089,4.611578,155.304333,6.0,4.0,2.0,479.5,549.5,16.00,696.0,708.5,139.25,726.5,763.5,26.50
13,20.137717,146.867848,12.994435,13.0,7.0,2.0,456.0,666.5,17.75,28.0,768.5,32.25,726.5,763.5,26.50
12,21.284867,73.547303,85.167830,12.0,1.0,4.0,262.0,26.5,15.75,68.5,248.5,14.50,696.0,708.5,139.25


In [ ]:
# STORED = []
# QUERY = np.zeros((N,15))
# for i in range(craters_det1.shape[0]):
#     triplet, HP, PICKS = find_triplet(df_craters_det1, idx = i)
#     Names = [triplet[0].name,triplet[1].name,triplet[2].name]
#     names = set(Names)
#     if names not in STORED:
#         STORED.append(names)
#         K_v = compute_K_vet(triplet)
#         c1 = [triplet[0].x,triplet[0].y,triplet[0].r]
#         c2 = [triplet[1].x,triplet[1].y,triplet[1].r]
#         c3 = [triplet[2].x,triplet[2].y,triplet[2].r]
#         QUERY[i] = np.hstack([K_v, Names,c1,c2,c3])
#     else:
#         triplet = find_other_triplet(triplet, STORED, PICKS, HP)
#         if triplet is not None:
#             Names = [triplet[0].name,triplet[1].name,triplet[2].name]
#             names = set(Names)
#             STORED.append(names)
#             K_v = compute_K_vet(triplet)
#             c1 = [triplet[0].x,triplet[0].y,triplet[0].r]
#             c2 = [triplet[1].x,triplet[1].y,triplet[1].r]
#             c3 = [triplet[2].x,triplet[2].y,triplet[2].r]
#             QUERY[i] = np.hstack([K_v, Names,c1,c2,c3])
    
#     printProgressBar(i+1, craters_det1.shape[0], printEnd='')
    
#     QUERY = pd.DataFrame(QUERY, columns=['Angle1','Angle2','Angle3','ctr1','ctr2','ctr3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])

In [11]:
def find_query(df_craters):
    STORED = []
    QUERY = np.zeros((df_craters.shape[0],15))
    for i in range(df_craters.shape[0]):
        triplet, HP, PICKS = find_triplet(df_craters, idx = i)
        Names = [triplet[0].name,triplet[1].name,triplet[2].name]
        names = set(Names)
        if names not in STORED:
            STORED.append(names)
            K_v = compute_K_vet(triplet)
            c1 = [triplet[0].x,triplet[0].y,triplet[0].r]
            c2 = [triplet[1].x,triplet[1].y,triplet[1].r]
            c3 = [triplet[2].x,triplet[2].y,triplet[2].r]
            QUERY[i] = np.hstack([K_v, Names,c1,c2,c3])
        else:
            triplet = find_other_triplet(triplet, STORED, PICKS, HP)
            if triplet is not None:
                Names = [triplet[0].name,triplet[1].name,triplet[2].name]
                names = set(Names)
                STORED.append(names)
                K_v = compute_K_vet(triplet)
                c1 = [triplet[0].x,triplet[0].y,triplet[0].r]
                c2 = [triplet[1].x,triplet[1].y,triplet[1].r]
                c3 = [triplet[2].x,triplet[2].y,triplet[2].r]
                QUERY[i] = np.hstack([K_v, Names,c1,c2,c3])

        printProgressBar(i+1, df_craters.shape[0], printEnd='')
    QUERY = pd.DataFrame(QUERY, columns=['Angle1','Angle2','Angle3','ctr1','ctr2','ctr3','x1','y1','r1','x2','y2','r2','x3','y3','r3'])
    return QUERY

In [ ]:
# # Searching:

# for i in QUERY1:
#     for j in QUERY2:
#         d = np.linalg.norm(i-j)
#         if d < 1:
#             print(i,j)

In [ ]:
# plt.figure(1, dpi=300)
# plt.subplot(121)
# img = img_plus_crts(img1, craters_det1)
# plt.imshow(img)

# plt.subplot(122)
# img = img_plus_crts(img2, craters_det2)
# plt.imshow(img)
# plt.show()

In [ ]:
# def matching(crat_det, crat_cat):
    
#     mat1 = crat_det
#     mat2 = crat_cat

#     MATCHER = []
#     for i in range(mat1.shape[0]):
#         for j in range(mat2.shape[0]):
#             f1 = mat1[i, :]
#             f2 = mat2[j, :]
#             tmp = sq_dif(f1,f2)
#             TMP = [i, j, tmp]
#             MATCHER.append(TMP)
#     flag = np.zeros(2)
#     for elem in MATCHER:
#         if elem[2] < 100:
#             match = [elem[0],elem[1]] 
#             flag = np.vstack((flag, match))
#     flag = flag[1:,:]
#     flag = np.array(flag).astype(int)
#     return flag


In [ ]:
# flag = np.zeros(2)
# for elem in MATCHER:
#     if elem[2] < 100:
#         match = [elem[0],elem[1]] 
#         flag = np.vstack((flag, match))
# flag = flag[1:,:]
# flag = np.array(flag).astype(int)
# flag

In [ ]:
# # Stack the images
# img1 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_1_Lat_0_Long_-147.8129.jpg')
# img2 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_2_Lat_0_Long_-147.7069.jpg')
# img_matched = np.hstack((img1,img2)).astype(int)
# plt.imshow(img_matched, cmap='gray')
# plt.axis(False)
# plt.grid(False)

In [ ]:
# cv2.arrowedLine(img_matched,start,end,(255, 0, 0) , 3)

In [ ]:
# center_coordinates = start
# radius = 500
# color = (0,0,255)
# thickness = 6
# cv2.circle(img_matched, center_coordinates, radius, color, thickness)